In [1]:
# -*- coding: utf-8 -*-
file_path = "./datas/hashcode_classification2020_train.csv"
import re
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
import numpy as np
from konlpy.tag import Okt
import json
import os
from pprint import pprint
import nltk
import keras
import pandas as pd

Using TensorFlow backend.


In [9]:
def read_data(file_path):
    datas = pd.read_csv(file_path, engine='python', names=['title', 'content', 'label'], encoding='utf-8')
    title = datas.loc[1:,'title'].values
    content = datas.loc[1:,'content'].values
    label = datas.loc[1:,'label'].values
    result_data = []
    for idx, title_data in enumerate(title):
        tmp_data = [title_data, content[idx], label[idx]]
        result_data.append(tmp_data)
    return result_data
read_data(file_path)

[['wait()과 sleep()의 차이점은 뭔가요',
  '###발생하는 문제 및 실행환경\nwait()과 sleep()의 차이점은 뭔가요',
  '3'],
 ['$$$ 초보 외부 클래스 멤버 변수 사용 질문합니다.ㅠㅠ',
  '1.헤더에, 사용할 멤버변수가 담긴 헤더 파일이 Include 되어있습니다.\n예를 들어 해당 클래스 이름이 xProtect 라면. \n가져와서 쓰고자 하는 현재 cpp 파일에 \n\nxProtect *A = NULL; 이나 xProtect *A = new xProtect\n\n를 사용하여 A라는 객체를 만들고, 사용하고자 하는 외부 xProtect 클래스의 멤버변수가 PP라는 리스트라면\n\nA->PP 와 같은 방법으로 외부클래스에서 저장된 리스트 PP를 불러오는 방식으로 사용하고싶은데요\n\n아무 방법도 먹히지 않고 해당 PP에는 쓰레기 값만 저장이 되어있네요..\n\n그냥 간단할 줄 알았는데 이게 하루를 잡아먹어 미칠 노릇입니다... \n\n다른 방법으로 객체화 해야되는건지... 아니면  PP라는 외부 클래스의 멤버변수를 가져올 수 있는 다른 방법이 있다면 알려주세요 ㅠㅠ 고수분들 답변 기다리겠습니다..  ',
  '2'],
 ['ORM: Sequelize: 다대다 관계 쿼리',
  "안녕하세요.\n\n어떻게 다대다 관계 쿼리를 해야하나요? 예를들어, `product`, `category`, `product_category` 모델이 있다고 하고 아래와 같은 association을 가지고 있습니다.\n\n```\n// product\nproduct.belongsToMany (models.category, {\n      through: 'product_category',\n      foreignKey: 'product_id'\n    });\n\n// category\ncategory.belongsToMany (models.product, {\n      through: 'product_category',\n      forei

In [3]:
def tokenize(doc):
    # norm은 정규화, stem은 근어로 표시하기를 나타냄
    okt = Okt()
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

In [4]:
def modeling(x_train, y_train):
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizers.RMSprop(lr=0.001),
                  loss=losses.binary_crossentropy,
                  metrics=[metrics.binary_accuracy])
    tb_hist = keras.callbacks.TensorBoard(log_dir='../graph', histogram_freq=0, write_graph=True,
                                          write_images=True)
    model.fit(x_train, y_train, epochs=25, batch_size=512, callbacks=[tb_hist])
    return model

In [5]:
def array_to_float(data):
    x_train = np.asarray(data).astype('float32')
    return x_train

In [6]:
def change_frequency(docs):
    train_x = [term_frequency(d) for d, _ in docs]
    train_y = [c for _, c in docs]
    return train_x, train_y

In [7]:
def make_selected_words(train_docs):
    tokens = [t for d in train_docs for t in d[0]]
    text = nltk.Text(tokens, name='NMSC')
    selected_words = [f[0] for f in text.vocab().most_common(10000)]
    return selected_words